## 使用微调后的 LLaMA2-7B 推理

In [1]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer


# model_dir = "models/llama-7-int4-dolly"
# model_dir = '/home/rr-ai/python-project/LLM-JK-quickstart/llama/models/llama-7-int4-dolly-20240313_094918/checkpoint-90'
model_dir = '/home/rr-ai/python-project/LLM-JK-quickstart/llama/models/Llama2-Chinese-13b-Chat-int4-finfq-20240314_170706/'

# 加载基础LLM模型与分词器
model = AutoPeftModelForCausalLM.from_pretrained(
    model_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
) 
tokenizer = AutoTokenizer.from_pretrained(model_dir)

/home/rr-ai/anaconda3/envs/fine_tuning_transformers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards:   0%|                                                            | 0/3 [00:00<?, ?it/s]/home/rr-ai/anaconda3/envs/fine_tuning_transformers/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.23s/it]
/home/rr-ai/anaconda3/envs/fine_tuning_transformers/

In [4]:
from datasets import load_dataset 
from random import randrange
 
 
# 从hub加载数据集并得到一个样本
# dataset = load_dataset("databricks/databricks-dolly-15k", split="train")
dataset = load_dataset("/home/rr-ai/huggingface_datasets/databricks-dolly-15k", split="train")

sample = dataset[randrange(len(dataset))]
 
prompt = f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM. 
 
### Input:
{sample['response']}
 
### Response:
"""
 
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()

outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.9)

print(f"Prompt:\n{sample['response']}\n")
print(f"Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"Ground truth:\n{sample['instruction']}")

Prompt:
Long hair, angled cut, beach waves

Generated instruction:
My advice would be to find a professional stylist. They will have a good idea of what kind of haircuts will work best for your hair and will be able to make it work for you. They will also be able to give you a better idea of what kinds of products will work best for your hair type. This may seem like a hassle but you will get better results that way.

Ground truth:
What are the best hairstyles for a round face?
